# Data Integration Pipeline for Kaggle Competition

## OpenAI to Z Challenge: Real Data Integration

This notebook demonstrates how to integrate real competition data from the Kaggle OpenAI to Z Challenge:

### Data Sources
1. **LiDAR Data**: Point cloud data in .las/.laz or GeoTIFF format
2. **Satellite Imagery**: High-resolution imagery (GeoTIFF, JPEG2000)
3. **NDVI Data**: Vegetation indices for land cover analysis
4. **GIS Data**: Terrain, land use, and archaeological context
5. **Archaeological Literature**: Research papers with DOI references

### Pipeline Stages
1. **Data Discovery & Download**
2. **Format Validation & Standardization**
3. **Coordinate System Alignment**
4. **Quality Assessment & Cleaning**
5. **Feature Extraction Pipeline**
6. **Integration with ML Models**

In [ ]:
# Import required libraries
import sys
import os
import json
import requests
import zipfile
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
# Note: Address specific warnings as they arise rather than suppressing all warnings

# Geospatial libraries
import rasterio
import geopandas as gpd
from rasterio.enums import Resampling
from rasterio.warp import calculate_default_transform, reproject
import fiona

# Add src directory to path
sys.path.append('../src')

# Import our custom modules
from data_loading import (
    LiDARLoader, SatelliteImageLoader, NDVILoader, 
    GISDataLoader, ArchaeologicalLiteratureLoader, DatasetValidator
)
from geospatial_processing import (
    CoordinateTransformer, RasterProcessor
)
from config import Config
from logging_utils import setup_logging

# Configuration
config = Config()
logger = setup_logging('INFO')

print("✅ All libraries imported successfully!")
print("🌍 Ready for real geospatial data integration!")
print(f"📍 Working directory: {Path.cwd()}")

## 1. Data Discovery & Setup

Set up directory structure and check for competition data availability.

In [ ]:
# Set up data directory structure
def setup_data_directories():
    """Create standardized directory structure for competition data."""
    
    base_dir = Path('../data')
    directories = {
        'raw': base_dir / 'raw',
        'processed': base_dir / 'processed',
        'lidar': base_dir / 'raw' / 'lidar',
        'satellite': base_dir / 'raw' / 'satellite',
        'ndvi': base_dir / 'raw' / 'ndvi',
        'gis': base_dir / 'raw' / 'gis',
        'literature': base_dir / 'raw' / 'literature',
        'aligned': base_dir / 'processed' / 'aligned',
        'features': base_dir / 'processed' / 'features',
        'models': base_dir / 'processed' / 'models'
    }
    
    print("📁 Setting up data directory structure...")
    for name, path in directories.items():
        path.mkdir(parents=True, exist_ok=True)
        print(f"   📂 {name}: {path}")
    
    return directories

# Create directories
data_dirs = setup_data_directories()

print(f"\n✅ Directory structure created successfully!")
print(f"📋 Ready to receive competition data in standardized format.")

In [ ]:
# Competition data checker
def check_competition_data_availability():
    """Check for available competition data and provide guidance."""
    
    print("🔍 Checking for Kaggle competition data...")
    
    # Expected file patterns for competition data
    expected_files = {
        'LiDAR': ['*.tif', '*.las', '*.laz'],
        'Satellite': ['*.tif', '*.jp2', '*.tiff'],
        'NDVI': ['*ndvi*.tif', '*vegetation*.tif'],
        'GIS': ['*.shp', '*.geojson', '*.gpkg'],
        'Literature': ['*.json', '*.csv', '*.txt']
    }
    
    data_status = {}
    
    for data_type, patterns in expected_files.items():
        data_dir = data_dirs['raw'] / data_type.lower()
        found_files = []
        
        for pattern in patterns:
            found_files.extend(list(data_dir.glob(pattern)))
        
        data_status[data_type] = {
            'available': len(found_files) > 0,
            'count': len(found_files),
            'files': [f.name for f in found_files[:3]]  # Show first 3
        }
        
        status_icon = "✅" if data_status[data_type]['available'] else "❌"
        print(f"   {status_icon} {data_type}: {data_status[data_type]['count']} files found")
        
        if data_status[data_type]['files']:
            print(f"      📄 Examples: {', '.join(data_status[data_type]['files'])}")
    
    # Overall status
    available_types = sum(1 for status in data_status.values() if status['available'])
    total_types = len(data_status)
    
    print(f"\n📊 Data Availability Summary: {available_types}/{total_types} data types available")
    
    if available_types == 0:
        print("\n📋 To add competition data:")
        print("   1. Download data from Kaggle competition page")
        print("   2. Extract files to appropriate subdirectories:")
        for data_type in expected_files.keys():
            print(f"      - {data_type}: {data_dirs['raw'] / data_type.lower()}")
        print("   3. Re-run this notebook to process real data")
        print("\n🔗 Competition URL: https://www.kaggle.com/competitions/openai-to-z-challenge/")
    
    return data_status

# Check data availability
data_status = check_competition_data_availability()

## 2. Sample Data Generator

Since competition data may not be immediately available, we'll create realistic sample data that follows expected formats.

In [ ]:
def create_sample_competition_data():
    """Create realistic sample data that mimics competition format."""
    
    print("🏗️  Creating sample competition data...")
    
    # Amazon basin coordinates (typical competition area)
    amazon_bounds = {
        'west': -75.0,
        'south': -10.0,
        'east': -45.0,
        'north': 5.0
    }
    
    # Sample area: 50km x 50km
    sample_area = {
        'west': -60.5,
        'south': -3.5,
        'east': -60.0,
        'north': -3.0
    }
    
    # Grid parameters
    resolution = 0.0001  # ~10m resolution
    width = int((sample_area['east'] - sample_area['west']) / resolution)
    height = int((sample_area['north'] - sample_area['south']) / resolution)
    
    print(f"   📏 Sample area: {width} x {height} pixels")
    print(f"   🌍 Coordinates: {sample_area}")
    print(f"   📐 Resolution: {resolution}° (~10m)")
    
    # Create coordinate arrays
    x_coords = np.linspace(sample_area['west'], sample_area['east'], width)
    y_coords = np.linspace(sample_area['south'], sample_area['north'], height)
    X, Y = np.meshgrid(x_coords, y_coords)
    
    # 1. LiDAR Elevation Data
    print("   🏔️  Generating LiDAR elevation data...")
    
    # Amazon-like elevation (0-200m with river valleys)
    base_elevation = 100
    elevation = (
        base_elevation +
        30 * np.sin(X * 500) * np.cos(Y * 400) +  # Large-scale terrain
        15 * np.sin(X * 1000) * np.sin(Y * 800) +  # Medium-scale features
        5 * np.random.normal(0, 1, (height, width))  # Small-scale noise
    )
    
    # Add river valleys (lower elevation)
    river_mask = np.abs(np.sin(X * 800)) < 0.1
    elevation[river_mask] -= 20
    
    elevation = np.clip(elevation, 0, 250).astype(np.float32)
    
    # 2. Satellite NDVI Data
    print("   🌱 Generating satellite NDVI data...")
    
    # Amazon vegetation patterns
    base_ndvi = 0.85  # Dense forest
    ndvi = (
        base_ndvi +
        0.1 * np.sin(X * 600) * np.cos(Y * 500) +  # Vegetation variations
        0.05 * np.random.normal(0, 1, (height, width))  # Natural variation
    )
    
    # Reduce NDVI near rivers (riparian zones)
    ndvi[river_mask] *= 0.7
    
    # Add deforested areas (archaeological signatures)
    for _ in range(8):  # 8 potential archaeological areas
        center_x = np.random.randint(width//4, 3*width//4)
        center_y = np.random.randint(height//4, 3*height//4)
        radius = np.random.randint(10, 30)
        
        # Create circular cleared area
        y_indices, x_indices = np.ogrid[:height, :width]
        mask = (x_indices - center_x)**2 + (y_indices - center_y)**2 <= radius**2
        ndvi[mask] -= np.random.uniform(0.2, 0.4)
    
    ndvi = np.clip(ndvi, 0, 1).astype(np.float32)
    
    # 3. Create sample files with proper geospatial metadata
    from rasterio.transform import from_bounds
    
    transform = from_bounds(
        sample_area['west'], sample_area['south'], 
        sample_area['east'], sample_area['north'],
        width, height
    )
    
    crs = 'EPSG:4326'  # WGS84
    
    # Save LiDAR elevation
    lidar_path = data_dirs['lidar'] / 'amazon_elevation_sample.tif'
    with rasterio.open(
        lidar_path, 'w',
        driver='GTiff',
        height=height, width=width,
        count=1, dtype=elevation.dtype,
        crs=crs, transform=transform,
        compress='lzw'
    ) as dst:
        dst.write(elevation, 1)
        dst.set_band_description(1, 'Elevation (meters)')
    
    # Save NDVI
    ndvi_path = data_dirs['ndvi'] / 'amazon_ndvi_sample.tif'
    with rasterio.open(
        ndvi_path, 'w',
        driver='GTiff',
        height=height, width=width,
        count=1, dtype=ndvi.dtype,
        crs=crs, transform=transform,
        compress='lzw'
    ) as dst:
        dst.write(ndvi, 1)
        dst.set_band_description(1, 'NDVI')
    
    # 4. Create sample archaeological literature data
    print("   📚 Generating archaeological literature data...")
    
    literature_data = {
        'metadata': {
            'source': 'Sample Archaeological Database',
            'region': 'Amazon Basin',
            'date_compiled': '2025-01-01',
            'coordinate_system': 'WGS84'
        },
        'papers': [
            {
                'doi': '10.1016/j.sample.2024.001',
                'title': 'Pre-Columbian Settlement Patterns in the Upper Amazon',
                'authors': ['Smith, J.', 'Garcia, M.', 'Silva, A.'],
                'year': 2024,
                'coordinates_mentioned': [
                    {'latitude': -3.25, 'longitude': -60.25, 'confidence': 0.9},
                    {'latitude': -3.15, 'longitude': -60.35, 'confidence': 0.7}
                ],
                'site_types': ['settlement', 'agricultural'],
                'cultural_period': '800-1200 CE'
            },
            {
                'doi': '10.1002/sample.2023.789',
                'title': 'LiDAR Detection of Archaeological Features in Amazonia',
                'authors': ['Brown, K.', 'Oliveira, P.'],
                'year': 2023,
                'coordinates_mentioned': [
                    {'latitude': -3.35, 'longitude': -60.15, 'confidence': 0.8}
                ],
                'site_types': ['ceremonial'],
                'cultural_period': '1000-1500 CE'
            }
        ]
    }
    
    literature_path = data_dirs['literature'] / 'archaeological_literature.json'
    with open(literature_path, 'w') as f:
        json.dump(literature_data, f, indent=2)
    
    # 5. Create sample GIS data (protected areas, rivers)
    print("   🗺️  Generating GIS vector data...")
    
    # Create sample river line
    from shapely.geometry import LineString, Point
    
    river_coords = [
        (sample_area['west'], sample_area['south'] + 0.1),
        (sample_area['west'] + 0.2, sample_area['south'] + 0.2),
        (sample_area['west'] + 0.4, sample_area['south'] + 0.15),
        (sample_area['east'], sample_area['north'] - 0.1)
    ]
    
    gis_data = gpd.GeoDataFrame({
        'feature_type': ['river'],
        'name': ['Rio Sample'],
        'geometry': [LineString(river_coords)]
    }, crs='EPSG:4326')
    
    gis_path = data_dirs['gis'] / 'amazon_features.geojson'
    gis_data.to_file(gis_path, driver='GeoJSON')
    
    print(f"✅ Sample data created successfully:")
    print(f"   🏔️  LiDAR: {lidar_path}")
    print(f"   🌱 NDVI: {ndvi_path}")
    print(f"   📚 Literature: {literature_path}")
    print(f"   🗺️  GIS: {gis_path}")
    
    return {
        'elevation_path': lidar_path,
        'ndvi_path': ndvi_path,
        'literature_path': literature_path,
        'gis_path': gis_path,
        'bounds': sample_area,
        'crs': crs,
        'resolution': resolution
    }

# Create sample data if no real data is available
if not any(status['available'] for status in data_status.values()):
    print("📋 No competition data found. Creating sample data for demonstration...")
    sample_data_info = create_sample_competition_data()
else:
    print("✅ Competition data detected. Will process real data.")
    sample_data_info = None

## 3. Data Loading & Validation Pipeline

Load and validate all available data sources.

In [ ]:
class DataIntegrationPipeline:
    """Comprehensive data integration pipeline for archaeological site detection."""
    
    def __init__(self, data_directories):
        self.data_dirs = data_directories
        self.validator = DatasetValidator()
        self.loaded_datasets = {}
        self.aligned_datasets = {}
        
    def discover_data_files(self):
        """Automatically discover data files in directory structure."""
        
        print("🔍 Discovering data files...")
        
        file_patterns = {
            'lidar': ['*.tif', '*.tiff', '*.las', '*.laz'],
            'satellite': ['*.tif', '*.tiff', '*.jp2'],
            'ndvi': ['*.tif', '*.tiff'],
            'gis': ['*.shp', '*.geojson', '*.gpkg'],
            'literature': ['*.json', '*.csv']
        }
        
        discovered_files = {}
        
        for data_type, patterns in file_patterns.items():
            data_dir = self.data_dirs[data_type]
            files = []
            
            for pattern in patterns:
                files.extend(list(data_dir.glob(pattern)))
            
            discovered_files[data_type] = files
            print(f"   📁 {data_type}: {len(files)} files found")
            
            for file_path in files[:3]:  # Show first 3 files
                print(f"      📄 {file_path.name}")
            
            if len(files) > 3:
                print(f"      ... and {len(files) - 3} more files")
        
        return discovered_files
    
    def load_raster_data(self, file_path):
        """Load and validate raster data."""
        
        try:
            with rasterio.open(file_path) as src:
                data = src.read(1).astype(np.float32)
                metadata = {
                    'crs': src.crs,
                    'transform': src.transform,
                    'bounds': src.bounds,
                    'shape': data.shape,
                    'dtype': data.dtype,
                    'nodata': src.nodata
                }
                
                # Data quality assessment
                quality = self.validator.assess_data_quality(data)
                
                print(f"   ✅ Loaded {file_path.name}:")
                print(f"      📏 Shape: {metadata['shape']}")
                print(f"      🗺️  CRS: {metadata['crs']}")
                print(f"      📊 Data range: {data.min():.3f} - {data.max():.3f}")
                print(f"      ✅ Quality score: {quality.quality_score:.3f}")
                
                return {
                    'data': data,
                    'metadata': metadata,
                    'quality': quality,
                    'source_path': file_path
                }
                
        except Exception as e:
            print(f"   ❌ Failed to load {file_path.name}: {e}")
            return None
    
    def load_vector_data(self, file_path):
        """Load and validate vector data."""
        
        try:
            gdf = gpd.read_file(file_path)
            
            print(f"   ✅ Loaded {file_path.name}:")
            print(f"      📊 Features: {len(gdf)}")
            print(f"      🗺️  CRS: {gdf.crs}")
            print(f"      📋 Columns: {list(gdf.columns)}")
            
            return {
                'data': gdf,
                'metadata': {
                    'crs': gdf.crs,
                    'bounds': gdf.total_bounds,
                    'feature_count': len(gdf)
                },
                'source_path': file_path
            }
            
        except Exception as e:
            print(f"   ❌ Failed to load {file_path.name}: {e}")
            return None
    
    def load_literature_data(self, file_path):
        """Load and validate archaeological literature data."""
        
        try:
            if file_path.suffix == '.json':
                with open(file_path, 'r') as f:
                    data = json.load(f)
            elif file_path.suffix == '.csv':
                data = pd.read_csv(file_path).to_dict('records')
            else:
                raise ValueError(f"Unsupported format: {file_path.suffix}")
            
            # Extract coordinate information
            coordinates = []
            if isinstance(data, dict) and 'papers' in data:
                for paper in data['papers']:
                    if 'coordinates_mentioned' in paper:
                        coordinates.extend(paper['coordinates_mentioned'])
            
            print(f"   ✅ Loaded {file_path.name}:")
            if isinstance(data, dict) and 'papers' in data:
                print(f"      📚 Papers: {len(data['papers'])}")
            print(f"      📍 Coordinates: {len(coordinates)}")
            
            return {
                'data': data,
                'coordinates': coordinates,
                'source_path': file_path
            }
            
        except Exception as e:
            print(f"   ❌ Failed to load {file_path.name}: {e}")
            return None
    
    def load_all_data(self):
        """Load all discovered data files."""
        
        print("📦 Loading all data files...")
        
        discovered_files = self.discover_data_files()
        
        # Load LiDAR data
        if discovered_files['lidar']:
            print("\n🏔️  Loading LiDAR data...")
            self.loaded_datasets['lidar'] = []
            for file_path in discovered_files['lidar']:
                dataset = self.load_raster_data(file_path)
                if dataset:
                    self.loaded_datasets['lidar'].append(dataset)
        
        # Load NDVI data
        if discovered_files['ndvi']:
            print("\n🌱 Loading NDVI data...")
            self.loaded_datasets['ndvi'] = []
            for file_path in discovered_files['ndvi']:
                dataset = self.load_raster_data(file_path)
                if dataset:
                    self.loaded_datasets['ndvi'].append(dataset)
        
        # Load satellite data
        if discovered_files['satellite']:
            print("\n🛰️  Loading satellite data...")
            self.loaded_datasets['satellite'] = []
            for file_path in discovered_files['satellite']:
                dataset = self.load_raster_data(file_path)
                if dataset:
                    self.loaded_datasets['satellite'].append(dataset)
        
        # Load GIS data
        if discovered_files['gis']:
            print("\n🗺️  Loading GIS data...")
            self.loaded_datasets['gis'] = []
            for file_path in discovered_files['gis']:
                dataset = self.load_vector_data(file_path)
                if dataset:
                    self.loaded_datasets['gis'].append(dataset)
        
        # Load literature data
        if discovered_files['literature']:
            print("\n📚 Loading literature data...")
            self.loaded_datasets['literature'] = []
            for file_path in discovered_files['literature']:
                dataset = self.load_literature_data(file_path)
                if dataset:
                    self.loaded_datasets['literature'].append(dataset)
        
        print(f"\n✅ Data loading complete:")
        for data_type, datasets in self.loaded_datasets.items():
            print(f"   📊 {data_type}: {len(datasets)} datasets loaded")
        
        return self.loaded_datasets

# Initialize pipeline
pipeline = DataIntegrationPipeline(data_dirs)

# Load all available data
loaded_data = pipeline.load_all_data()

## 4. Coordinate System Alignment

Align all datasets to a common coordinate reference system and spatial extent.

In [ ]:
def align_datasets_to_common_grid(loaded_datasets, target_crs='EPSG:4326', target_resolution=0.0001):
    """Align all raster datasets to a common grid."""
    
    print(f"🔄 Aligning datasets to common grid...")
    print(f"   🎯 Target CRS: {target_crs}")
    print(f"   📐 Target resolution: {target_resolution}° (~10m)")
    
    # Collect all raster datasets
    raster_datasets = []
    for data_type in ['lidar', 'ndvi', 'satellite']:
        if data_type in loaded_datasets:
            raster_datasets.extend(loaded_datasets[data_type])
    
    if not raster_datasets:
        print("❌ No raster datasets found for alignment")
        return {}
    
    print(f"   📊 Aligning {len(raster_datasets)} raster datasets")
    
    # Find common bounds
    all_bounds = []
    for dataset in raster_datasets:
        bounds = dataset['metadata']['bounds']
        
        # Transform bounds to target CRS if needed
        src_crs = dataset['metadata']['crs']
        if src_crs != target_crs:
            transformer = CoordinateTransformer(str(src_crs), target_crs)
            transformed_bounds = transformer.transform_bounds(bounds)
            all_bounds.append(transformed_bounds)
        else:
            all_bounds.append(bounds)
    
    # Calculate intersection of all bounds
    common_bounds = (
        max(bounds[0] for bounds in all_bounds),  # west
        max(bounds[1] for bounds in all_bounds),  # south
        min(bounds[2] for bounds in all_bounds),  # east
        min(bounds[3] for bounds in all_bounds)   # north
    )
    
    print(f"   🌍 Common bounds: {common_bounds}")
    
    # Check if common bounds are valid
    if common_bounds[0] >= common_bounds[2] or common_bounds[1] >= common_bounds[3]:
        print("❌ No spatial overlap between datasets")
        return {}
    
    # Calculate common grid dimensions
    width = int((common_bounds[2] - common_bounds[0]) / target_resolution)
    height = int((common_bounds[3] - common_bounds[1]) / target_resolution)
    
    print(f"   📏 Common grid: {width} x {height} pixels")
    
    # Create common transform
    from rasterio.transform import from_bounds
    common_transform = from_bounds(
        common_bounds[0], common_bounds[1],
        common_bounds[2], common_bounds[3],
        width, height
    )
    
    aligned_datasets = {}
    
    # Align each dataset
    for i, dataset in enumerate(raster_datasets):
        print(f"   📊 Aligning dataset {i+1}/{len(raster_datasets)}...")
        
        src_data = dataset['data']
        src_transform = dataset['metadata']['transform']
        src_crs = dataset['metadata']['crs']
        
        # Create output array
        aligned_data = np.zeros((height, width), dtype=np.float32)
        
        # Reproject to common grid
        reproject(
            source=src_data,
            destination=aligned_data,
            src_transform=src_transform,
            src_crs=src_crs,
            dst_transform=common_transform,
            dst_crs=target_crs,
            resampling=Resampling.bilinear
        )
        
        # Store aligned dataset
        aligned_key = f"aligned_{i}"
        aligned_datasets[aligned_key] = {
            'data': aligned_data,
            'metadata': {
                'crs': target_crs,
                'transform': common_transform,
                'bounds': common_bounds,
                'shape': (height, width)
            },
            'original_dataset': dataset
        }
    
    print(f"✅ Dataset alignment complete: {len(aligned_datasets)} datasets aligned")
    
    return aligned_datasets

# Align datasets if we have raster data
if any(data_type in loaded_data for data_type in ['lidar', 'ndvi', 'satellite']):
    aligned_data = align_datasets_to_common_grid(loaded_data)
    
    if aligned_data:
        # Save aligned datasets
        print("\n💾 Saving aligned datasets...")
        
        for key, dataset in aligned_data.items():
            output_path = data_dirs['aligned'] / f'{key}.tif'
            
            with rasterio.open(
                output_path, 'w',
                driver='GTiff',
                height=dataset['metadata']['shape'][0],
                width=dataset['metadata']['shape'][1],
                count=1,
                dtype=dataset['data'].dtype,
                crs=dataset['metadata']['crs'],
                transform=dataset['metadata']['transform'],
                compress='lzw'
            ) as dst:
                dst.write(dataset['data'], 1)
            
            print(f"   💾 Saved: {output_path.name}")
    
else:
    print("⚠️  No raster datasets available for alignment")
    aligned_data = {}

## 5. Data Visualization & Quality Assessment

Visualize loaded and aligned data for quality control.

In [ ]:
def visualize_integrated_data(aligned_data, loaded_data):
    """Visualize integrated datasets for quality assessment."""
    
    if not aligned_data:
        print("⚠️  No aligned data available for visualization")
        return
    
    print("📊 Creating data visualization...")
    
    # Determine number of datasets to visualize
    n_datasets = len(aligned_data)
    
    if n_datasets == 0:
        print("❌ No datasets to visualize")
        return
    
    # Create subplot layout
    cols = min(3, n_datasets)
    rows = (n_datasets + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 4*rows))
    
    if rows == 1 and cols == 1:
        axes = [axes]
    elif rows == 1:
        axes = axes
    else:
        axes = axes.flatten()
    
    # Plot each aligned dataset
    for i, (key, dataset) in enumerate(aligned_data.items()):
        data = dataset['data']
        original = dataset['original_dataset']
        
        # Determine data type and color scheme
        source_name = original['source_path'].stem
        if 'elevation' in source_name.lower() or 'lidar' in source_name.lower():
            cmap = 'terrain'
            title = f'Elevation Data\n{source_name}'
            unit = 'm'
        elif 'ndvi' in source_name.lower():
            cmap = 'RdYlGn'
            title = f'NDVI Data\n{source_name}'
            unit = ''
        else:
            cmap = 'viridis'
            title = f'Satellite Data\n{source_name}'
            unit = ''
        
        # Plot data
        im = axes[i].imshow(data, cmap=cmap, aspect='equal')
        axes[i].set_title(title, fontweight='bold', fontsize=10)
        axes[i].set_xlabel('X Coordinate')
        axes[i].set_ylabel('Y Coordinate')
        
        # Add colorbar
        plt.colorbar(im, ax=axes[i], shrink=0.8, label=unit)
        
        # Add data statistics
        stats_text = f'Range: {data.min():.2f} - {data.max():.2f}\nMean: {data.mean():.2f}'
        axes[i].text(0.02, 0.98, stats_text, transform=axes[i].transAxes, 
                    verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
                    fontsize=8)
    
    # Hide unused subplots
    for i in range(len(aligned_data), len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    plt.show()
    
    # Show spatial overlap information
    if aligned_data:
        sample_dataset = next(iter(aligned_data.values()))
        bounds = sample_dataset['metadata']['bounds']
        shape = sample_dataset['metadata']['shape']
        
        print(f"\n📊 Aligned Data Summary:")
        print(f"   🌍 Spatial Extent: {bounds[0]:.4f}, {bounds[1]:.4f} to {bounds[2]:.4f}, {bounds[3]:.4f}")
        print(f"   📏 Grid Size: {shape[1]} x {shape[0]} pixels")
        print(f"   📐 Coverage: {(bounds[2] - bounds[0]):.4f}° x {(bounds[3] - bounds[1]):.4f}°")
        print(f"   🎯 Pixel Size: ~{(bounds[2] - bounds[0]) / shape[1] * 111:.0f}m x {(bounds[3] - bounds[1]) / shape[0] * 111:.0f}m")
    
    # Literature data summary
    if 'literature' in loaded_data:
        print(f"\n📚 Literature Data Summary:")
        total_coords = 0
        for lit_dataset in loaded_data['literature']:
            coords = lit_dataset['coordinates']
            total_coords += len(coords)
            print(f"   📍 {len(coords)} coordinate references found")
            
            # Show sample coordinates
            for coord in coords[:3]:
                print(f"      📌 ({coord['latitude']:.3f}, {coord['longitude']:.3f}) - confidence: {coord['confidence']:.2f}")
        
        print(f"   📊 Total literature coordinates: {total_coords}")
    
    print(f"\n✅ Data visualization complete!")

# Visualize the integrated data
visualize_integrated_data(aligned_data, loaded_data)

## 6. Feature Extraction Pipeline

Extract features from integrated datasets for machine learning.

In [ ]:
def extract_features_from_aligned_data(aligned_data):
    """Extract comprehensive features from aligned datasets."""
    
    if not aligned_data:
        print("⚠️  No aligned data available for feature extraction")
        return None
    
    print("🔍 Extracting features from aligned datasets...")
    
    # Initialize feature extraction components
    from geospatial_processing import TerrainAnalyzer, VegetationAnalyzer, SpatialFeatureExtractor
    
    terrain_analyzer = TerrainAnalyzer()
    vegetation_analyzer = VegetationAnalyzer()
    feature_extractor = SpatialFeatureExtractor()
    
    # Identify data types from aligned datasets
    elevation_data = None
    ndvi_data = None
    satellite_data = None
    
    for key, dataset in aligned_data.items():
        source_name = dataset['original_dataset']['source_path'].stem.lower()
        data = dataset['data']
        
        if 'elevation' in source_name or 'lidar' in source_name:
            elevation_data = data
            print(f"   🏔️  Elevation data identified: {source_name}")
        elif 'ndvi' in source_name:
            ndvi_data = data
            print(f"   🌱 NDVI data identified: {source_name}")
        else:
            satellite_data = data
            print(f"   🛰️  Satellite data identified: {source_name}")
    
    extracted_features = {}
    
    # Extract terrain features from elevation data
    if elevation_data is not None:
        print(f"   🏔️  Extracting terrain features...")
        
        pixel_size = 10  # Approximate 10m resolution
        
        slope = terrain_analyzer.calculate_slope(elevation_data, pixel_size)
        aspect = terrain_analyzer.calculate_aspect(elevation_data, pixel_size)
        curvature = terrain_analyzer.calculate_curvature(elevation_data, pixel_size)
        terrain_features = terrain_analyzer.identify_terrain_features(elevation_data)
        
        extracted_features.update({
            'elevation': elevation_data,
            'slope': slope,
            'aspect': aspect,
            'profile_curvature': curvature['profile_curvature'],
            'plan_curvature': curvature['plan_curvature'],
            'terrain_features': terrain_features
        })
        
        print(f"      ✅ Terrain features extracted: elevation, slope, aspect, curvature")
        print(f"      🗻 Terrain features identified: {len(terrain_features['peaks'])} peaks, {len(terrain_features['valleys'])} valleys")
    
    # Extract vegetation features from NDVI data
    if ndvi_data is not None:
        print(f"   🌱 Extracting vegetation features...")
        
        vegetation_classes = vegetation_analyzer.classify_vegetation_density(ndvi_data)
        vegetation_anomalies = vegetation_analyzer.detect_vegetation_anomalies(
            ndvi_data, window_size=3, threshold=0.1
        )
        
        # Create synthetic NIR and Red bands for additional indices
        red_band = np.random.uniform(0.1, 0.3, ndvi_data.shape)
        nir_band = red_band * (1 + ndvi_data) / (1 - ndvi_data + 1e-6)  # Correctly derive NIR from NDVI formula: NIR = Red * (1 + NDVI) / (1 - NDVI)
        vegetation_indices = vegetation_analyzer.calculate_vegetation_indices(red_band, nir_band)
        
        extracted_features.update({
            'ndvi': ndvi_data,
            'vegetation_classes': vegetation_classes,
            'vegetation_anomalies': vegetation_anomalies.astype(float),
            'savi': vegetation_indices['savi'],
            'evi': vegetation_indices['evi']
        })
        
        print(f"      ✅ Vegetation features extracted: NDVI, classes, anomalies, SAVI, EVI")
        print(f"      ⚠️  Vegetation anomalies: {np.sum(vegetation_anomalies)} pixels ({np.sum(vegetation_anomalies)/vegetation_anomalies.size*100:.2f}%)")
    
    # Extract spatial/textural features
    if elevation_data is not None:
        print(f"   🏗️  Extracting spatial features...")
        
        # Sample for textural analysis (computationally intensive)
        step = max(1, elevation_data.shape[0] // 200)  # Sample to ~200x200 for speed
        elevation_sample = elevation_data[::step, ::step]
        
        textural_features_sample = feature_extractor.extract_textural_features(
            elevation_sample, window_size=3
        )
        
        # Upsample back to original size
        from scipy.ndimage import zoom
        zoom_factor = (elevation_data.shape[0] / textural_features_sample['contrast'].shape[0],
                      elevation_data.shape[1] / textural_features_sample['contrast'].shape[1])
        
        textural_features = {}
        for key, feature in textural_features_sample.items():
            textural_features[key] = zoom(feature, zoom_factor, order=1)
        
        extracted_features.update({
            'texture_contrast': textural_features['contrast'],
            'texture_homogeneity': textural_features['homogeneity'],
            'texture_energy': textural_features['energy'],
            'texture_dissimilarity': textural_features['dissimilarity']
        })
        
        print(f"      ✅ Spatial features extracted: {list(textural_features.keys())}")
    
    # Create composite features
    if elevation_data is not None and ndvi_data is not None:
        print(f"   🎯 Creating composite features...")
        
        # Archaeological preference score
        archaeological_preference = (
            np.exp(-((elevation_data - 150)**2) / (2 * 75**2)) *  # Elevation preference
            np.exp(-(slope**2) / (2 * 10**2)) *  # Slope preference
            (1 + extracted_features['texture_contrast'] / extracted_features['texture_contrast'].max())  # Terrain distinctiveness
        )
        
        # Terrain roughness
        terrain_roughness = np.sqrt(
            slope**2 + 
            extracted_features['profile_curvature']**2 + 
            extracted_features['plan_curvature']**2
        )
        
        # Vegetation health
        vegetation_health = ndvi_data * (1 - extracted_features['vegetation_anomalies'])
        
        extracted_features.update({
            'archaeological_preference': archaeological_preference,
            'terrain_roughness': terrain_roughness,
            'vegetation_health': vegetation_health
        })
        
        print(f"      ✅ Composite features created: archaeological_preference, terrain_roughness, vegetation_health")
    
    print(f"\n✅ Feature extraction complete:")
    print(f"   📊 Total features: {len(extracted_features)}")
    print(f"   📋 Feature list: {list(extracted_features.keys())}")
    
    # Save features to disk
    print(f"\n💾 Saving extracted features...")
    
    for feature_name, feature_data in extracted_features.items():
        if isinstance(feature_data, np.ndarray):
            # Save as GeoTIFF with same georeferencing as aligned data
            sample_dataset = next(iter(aligned_data.values()))
            metadata = sample_dataset['metadata']
            
            output_path = data_dirs['features'] / f'{feature_name}.tif'
            
            with rasterio.open(
                output_path, 'w',
                driver='GTiff',
                height=metadata['shape'][0],
                width=metadata['shape'][1],
                count=1,
                dtype=feature_data.dtype,
                crs=metadata['crs'],
                transform=metadata['transform'],
                compress='lzw'
            ) as dst:
                dst.write(feature_data, 1)
                dst.set_band_description(1, feature_name)
            
            print(f"   💾 {feature_name}: {output_path.name}")
    
    return extracted_features

# Extract features from aligned data
if aligned_data:
    extracted_features = extract_features_from_aligned_data(aligned_data)
else:
    print("⚠️  No aligned data available for feature extraction")
    extracted_features = None

## 7. Integration with Literature Data

Integrate archaeological literature coordinates with spatial data.

In [ ]:
def integrate_literature_coordinates(extracted_features, loaded_data, aligned_data):
    """Integrate archaeological literature coordinates with spatial features."""
    
    if 'literature' not in loaded_data or not loaded_data['literature']:
        print("⚠️  No literature data available for integration")
        return None
    
    if not extracted_features or not aligned_data:
        print("⚠️  No spatial features available for literature integration")
        return None
    
    print("📚 Integrating literature coordinates with spatial data...")
    
    # Get spatial reference information
    sample_dataset = next(iter(aligned_data.values()))
    bounds = sample_dataset['metadata']['bounds']
    transform = sample_dataset['metadata']['transform']
    shape = sample_dataset['metadata']['shape']
    
    print(f"   🌍 Spatial bounds: {bounds}")
    print(f"   📏 Grid shape: {shape}")
    
    # Collect all literature coordinates
    all_literature_coords = []
    
    for lit_dataset in loaded_data['literature']:
        coords = lit_dataset['coordinates']
        all_literature_coords.extend(coords)
    
    print(f"   📍 Total literature coordinates: {len(all_literature_coords)}")
    
    # Filter coordinates within spatial bounds
    valid_coords = []
    
    for coord in all_literature_coords:
        lat, lon = coord['latitude'], coord['longitude']
        
        if (bounds[0] <= lon <= bounds[2] and 
            bounds[1] <= lat <= bounds[3]):
            
            # Convert to pixel coordinates
            col = int((lon - bounds[0]) / (bounds[2] - bounds[0]) * shape[1])
            row = int((bounds[3] - lat) / (bounds[3] - bounds[1]) * shape[0])
            
            # Ensure within bounds
            if 0 <= row < shape[0] and 0 <= col < shape[1]:
                coord_info = {
                    'latitude': lat,
                    'longitude': lon,
                    'pixel_row': row,
                    'pixel_col': col,
                    'confidence': coord['confidence']
                }
                valid_coords.append(coord_info)
    
    print(f"   ✅ Coordinates within study area: {len(valid_coords)}")
    
    if not valid_coords:
        print("   ⚠️  No literature coordinates overlap with spatial data")
        return None
    
    # Extract feature values at literature coordinate locations
    literature_features = []
    
    for coord in valid_coords:
        row, col = coord['pixel_row'], coord['pixel_col']
        
        feature_values = {
            'latitude': coord['latitude'],
            'longitude': coord['longitude'],
            'confidence': coord['confidence'],
            'pixel_row': row,
            'pixel_col': col
        }
        
        # Extract feature values at this location
        for feature_name, feature_data in extracted_features.items():
            if isinstance(feature_data, np.ndarray) and feature_data.ndim == 2:
                feature_values[feature_name] = float(feature_data[row, col])
        
        literature_features.append(feature_values)
    
    # Create DataFrame for analysis
    literature_df = pd.DataFrame(literature_features)
    
    print(f"\n📊 Literature-Spatial Integration Summary:")
    print(f"   📚 Literature sites in study area: {len(literature_df)}")
    print(f"   📋 Features extracted per site: {len([col for col in literature_df.columns if col not in ['latitude', 'longitude', 'confidence', 'pixel_row', 'pixel_col']])}")
    
    # Show feature statistics at literature sites
    if 'elevation' in literature_df.columns:
        print(f"   🏔️  Elevation range at sites: {literature_df['elevation'].min():.1f} - {literature_df['elevation'].max():.1f}m")
    if 'ndvi' in literature_df.columns:
        print(f"   🌱 NDVI range at sites: {literature_df['ndvi'].min():.3f} - {literature_df['ndvi'].max():.3f}")
    if 'slope' in literature_df.columns:
        print(f"   📐 Slope range at sites: {literature_df['slope'].min():.1f} - {literature_df['slope'].max():.1f}°")
    
    # Display sample of literature sites
    print(f"\n📋 Sample Literature Sites:")
    display_cols = ['latitude', 'longitude', 'confidence']
    if 'elevation' in literature_df.columns:
        display_cols.append('elevation')
    if 'ndvi' in literature_df.columns:
        display_cols.append('ndvi')
    
    sample_df = literature_df[display_cols].head()
    print(sample_df.round(3).to_string(index=False))
    
    # Save literature integration results
    output_path = data_dirs['processed'] / 'literature_spatial_integration.csv'
    literature_df.to_csv(output_path, index=False)
    print(f"\n💾 Literature integration saved: {output_path.name}")
    
    return literature_df

# Integrate literature data
if extracted_features and 'literature' in loaded_data:
    literature_integration = integrate_literature_coordinates(
        extracted_features, loaded_data, aligned_data
    )
else:
    print("⚠️  Cannot integrate literature data: missing features or literature data")
    literature_integration = None

## 8. Data Integration Summary & Export

Summarize the data integration pipeline and prepare for ML model training.

In [ ]:
# Create comprehensive summary of data integration pipeline
print("📋 DATA INTEGRATION PIPELINE SUMMARY")
print("=" * 60)

print(f"\n📁 DIRECTORY STRUCTURE:")
for name, path in data_dirs.items():
    file_count = len(list(path.glob('*'))) if path.exists() else 0
    print(f"   📂 {name}: {path} ({file_count} files)")

print(f"\n📦 DATA LOADING RESULTS:")
total_datasets = 0
for data_type, datasets in loaded_data.items():
    count = len(datasets) if datasets else 0
    total_datasets += count
    status_icon = "✅" if count > 0 else "❌"
    print(f"   {status_icon} {data_type}: {count} datasets loaded")

print(f"   📊 Total datasets loaded: {total_datasets}")

print(f"\n🔄 DATA ALIGNMENT:")
if aligned_data:
    sample_dataset = next(iter(aligned_data.values()))
    bounds = sample_dataset['metadata']['bounds']
    shape = sample_dataset['metadata']['shape']
    
    print(f"   ✅ Alignment successful: {len(aligned_data)} datasets aligned")
    print(f"   🌍 Common extent: {bounds[0]:.4f}, {bounds[1]:.4f} to {bounds[2]:.4f}, {bounds[3]:.4f}")
    print(f"   📏 Grid dimensions: {shape[1]} x {shape[0]} pixels")
    print(f"   📐 Approximate resolution: {(bounds[2] - bounds[0]) / shape[1] * 111:.0f}m x {(bounds[3] - bounds[1]) / shape[0] * 111:.0f}m")
    print(f"   🎯 CRS: {sample_dataset['metadata']['crs']}")
else:
    print(f"   ❌ No raster data alignment performed")

print(f"\n🔍 FEATURE EXTRACTION:")
if extracted_features:
    print(f"   ✅ Feature extraction successful: {len(extracted_features)} features")
    
    # Categorize features
    terrain_features = [f for f in extracted_features.keys() if any(t in f for t in ['elevation', 'slope', 'aspect', 'curvature'])]
    vegetation_features = [f for f in extracted_features.keys() if any(v in f for v in ['ndvi', 'vegetation', 'savi', 'evi'])]
    texture_features = [f for f in extracted_features.keys() if 'texture' in f]
    composite_features = [f for f in extracted_features.keys() if any(c in f for c in ['archaeological', 'roughness', 'health'])]
    
    print(f"   🏔️  Terrain features ({len(terrain_features)}): {', '.join(terrain_features)}")
    print(f"   🌱 Vegetation features ({len(vegetation_features)}): {', '.join(vegetation_features)}")
    print(f"   🏗️  Texture features ({len(texture_features)}): {', '.join(texture_features)}")
    print(f"   🎯 Composite features ({len(composite_features)}): {', '.join(composite_features)}")
else:
    print(f"   ❌ No features extracted")

print(f"\n📚 LITERATURE INTEGRATION:")
if literature_integration is not None:
    print(f"   ✅ Literature integration successful: {len(literature_integration)} sites")
    print(f"   📍 Coordinate range:")
    print(f"      Latitude: {literature_integration['latitude'].min():.4f} to {literature_integration['latitude'].max():.4f}")
    print(f"      Longitude: {literature_integration['longitude'].min():.4f} to {literature_integration['longitude'].max():.4f}")
    print(f"   🎯 Confidence range: {literature_integration['confidence'].min():.2f} to {literature_integration['confidence'].max():.2f}")
else:
    print(f"   ❌ No literature integration performed")

print(f"\n📊 DATA QUALITY ASSESSMENT:")
if aligned_data and extracted_features:
    # Basic quality metrics
    valid_pixels = 0
    total_pixels = 0
    
    for feature_name, feature_data in extracted_features.items():
        if isinstance(feature_data, np.ndarray) and feature_data.ndim == 2:
            total_pixels = feature_data.size
            valid_pixels = np.sum(~np.isnan(feature_data) & ~np.isinf(feature_data))
            break
    
    if total_pixels > 0:
        completeness = valid_pixels / total_pixels * 100
        print(f"   ✅ Data completeness: {completeness:.1f}% ({valid_pixels}/{total_pixels} valid pixels)")
    
    print(f"   🎯 Ready for machine learning model training")
else:
    print(f"   ⚠️  Limited data available for quality assessment")

print(f"\n🚀 NEXT STEPS:")
print(f"   1. 🤖 Load data in machine learning notebooks")
print(f"   2. 📊 Create training/validation datasets")
print(f"   3. 🎯 Train archaeological site detection models")
print(f"   4. ✅ Validate predictions against literature sites")
print(f"   5. 📋 Generate competition submission")

print(f"\n💾 OUTPUT FILES:")
output_files = {
    'aligned_data': list(data_dirs['aligned'].glob('*.tif')),
    'features': list(data_dirs['features'].glob('*.tif')),
    'integration': list(data_dirs['processed'].glob('*.csv'))
}

for category, files in output_files.items():
    print(f"   📁 {category}: {len(files)} files")
    for file_path in files[:3]:
        print(f"      📄 {file_path.name}")
    if len(files) > 3:
        print(f"      ... and {len(files) - 3} more files")

print("\n" + "=" * 60)
print("🎯 DATA INTEGRATION PIPELINE COMPLETED SUCCESSFULLY ✅")
print("=" * 60)

# Create pipeline status report
pipeline_status = {
    'timestamp': pd.Timestamp.now().isoformat(),
    'total_datasets_loaded': total_datasets,
    'aligned_datasets': len(aligned_data) if aligned_data else 0,
    'extracted_features': len(extracted_features) if extracted_features else 0,
    'literature_sites': len(literature_integration) if literature_integration is not None else 0,
    'spatial_bounds': bounds if aligned_data else None,
    'grid_shape': shape if aligned_data else None,
    'ready_for_ml': bool(extracted_features and aligned_data)
}

# Save pipeline status
status_path = data_dirs['processed'] / 'pipeline_status.json'
with open(status_path, 'w') as f:
    json.dump(pipeline_status, f, indent=2, default=str)

print(f"\n💾 Pipeline status saved: {status_path.name}")
print(f"✅ Data integration pipeline ready for machine learning workflows!")